<p style="color:white; background-color: #366092; font-size: 15pt; padding: 5px 0px 5px 10px; font-weight: bold; text-align: left">  Totaliza lucros e prejuízos  </p>

In [ ]:
# Agrupa por mês totalizando o lucro/Prejuízo
df_sum_lucro_perda = dfFinal.groupby(dfFinal['Data da Operação']
                                     .dt.strftime('%B'))['Resultado (R$)'].sum()

In [ ]:
# Totaliza lucros
df_lucros = dfFinal.loc[dfFinal['Resultado (R$)'] > 0] \
                   .groupby(dfFinal['Data da Operação']
                   .dt.strftime('%B'))['Resultado (R$)']
df_sum_lucros = df_lucros.sum()
df_qt_lucros  = df_lucros.count()

# Totaliza Prejuízos
df_perdas = dfFinal.loc[dfFinal['Resultado (R$)'] < 0] \
                   .groupby(dfFinal['Data da Operação']
                   .dt.strftime('%B'))['Resultado (R$)']
df_sum_perdas = df_perdas.sum()
df_qt_perdas  = df_perdas.count()

In [ ]:
# Agrupa por mês totalizando a Porcentagem
df_sum_porcentagem = dfFinal.groupby(dfFinal['Data da Operação']
                                     .dt.strftime('%B'))['%'].sum()

In [ ]:
dfFinal_teste = dfFinal.groupby(['Data da Operação', 'Tipo Operação']).sum(['Resultado (R$)', 'Valor Compra'])
dfFinal_teste.reset_index(drop=False, inplace=True)

In [ ]:
# del dfFinal_teste['Preço (R$)']
del dfFinal_teste['%']
dfFinal_teste['%'] = (dfFinal_teste['Resultado (R$)'] * 100) / dfFinal_teste['Valor Compra']

In [ ]:
dfFinal_teste['Data da Operação'] = pd.to_datetime(dfFinal_teste['Data da Operação'], format='%d/%m/%Y')
df_sum_porcentagem_teste = dfFinal_teste.groupby(dfFinal_teste['Data da Operação']
                                        .dt.strftime('%B'))['%'].sum()

In [ ]:
array_meses = [4,8,12,2,1,7,6,3,5,11,10,9]
meses = []
if date.today().year == ano:
    for x in array_meses:
        if x <= date.today().month:
            meses.append(x)
else:
    meses = array_meses

In [ ]:
# corrige lista de meses se mês atual não tiver operações
if len(meses) > len(df_sum_lucro_perda):
    meses.remove(max(meses))

In [ ]:
# Cria DataFrame
df_tot_mes = pd.DataFrame({'mes':meses
                         ,'Mês':df_sum_lucro_perda.index.values
                         ,'Resultado (R$)':df_sum_lucro_perda
                         ,'Porcentagem':df_sum_porcentagem
                         ,'Total Lucros':df_sum_lucros
                         ,'Total Perdas':df_sum_perdas
                         ,'Qt Lucros':df_qt_lucros
                         ,'Qt Perdas':df_qt_perdas
                          }).set_index('mes')

In [ ]:
df_tot_mes['Total Lucros'] = df_tot_mes['Total Lucros'].fillna(0.00)
df_tot_mes['Total Perdas'] = df_tot_mes['Total Perdas'].fillna(0.00)
df_tot_mes['Qt Lucros'] = df_tot_mes['Qt Lucros'].fillna(0).astype(int)
df_tot_mes['Qt Perdas'] = df_tot_mes['Qt Perdas'].fillna(0).astype(int)

In [ ]:
df_tot_mes.sort_index(inplace=True)

In [ ]:
# Setando separador de milhar
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# Calculando total do ano
resultado = round(df_sum['Resultado (R$)'].sum(), 2)
porcentagem = round(df_sum['Porcentagem'].sum(), 2)
total_lucros = round(df_tot_mes['Total Lucros'].sum(), 2)
total_perdas = round(df_tot_mes['Total Perdas'].sum(), 2)
qt_lucros = round(df_tot_mes['Qt Lucros'].sum(), 2)
qt_perdas = round(df_tot_mes['Qt Perdas'].sum(), 2)
resultado_txt = 'Lucro: ' if resultado >= 0 else 'Prejuízo...: '
porcentagem_txt = '> Porcentagem: '
# Markdown(f'{porcentagem_txt}**{porcentagem}%**<br>{resultado_txt}**R${resultado}**')

In [ ]:
# Adicionando Total do Ano
df_tot_mes.loc[len(df_tot_mes) + 1] = ['== Ano ==', resultado, porcentagem, total_lucros, total_perdas, qt_lucros, qt_perdas]
df_tot_mes.rename(index={len(df_tot_mes) : 'Total'}, inplace=True)

In [ ]:
# Calcula mediana to valor total usado para compras
teste = dfFinal.loc[dfFinal["Tipo Operação"] == "Compra"].drop(columns=["Ativo"])
teste['AnoMes'] = teste['Data da Operação'].dt.to_period('M')
resultado = teste.groupby('AnoMes')['Valor Compra'].agg(['mean', 'median']).reset_index()
resultado.loc[len(resultado) + 1] = [resultado["AnoMes"][0], 0, resultado["median"].median()]
resultado = resultado.reset_index(drop=True)
resultado.index += 1
resultado.rename(index={len(resultado) : 'Total'}, inplace=True)
df_tot_mes["Mediana Compras"] = resultado["median"]
df_tot_mes["Porcentagem Relatório"] = df_tot_mes["Porcentagem"]
df_tot_mes["Porcentagem"] = df_tot_mes["Resultado (R$)"] / df_tot_mes["Mediana Compras"]

In [ ]:
# teste
# df_tot_mes = df_tot_mes.style.set_table_styles([dict(selector="th",props=[('max-width', '90px'), ('text-align', 'center')])])

In [ ]:
df_tot_mes_format = df_tot_mes.copy()
df_tot_mes_format["Porcentagem Relatório"] = df_tot_mes_format["Porcentagem Relatório"] * 0.01
df_tot_mes_format = df_tot_mes_format.style.applymap(color_negative_red, subset=['Resultado (R$)'
                                                                                 ,'Porcentagem'
                                                                                 ,'Total Lucros'
                                                                                 ,'Total Perdas'
                                                                                 ,'Mediana Compras'
                                                                                 ,'Porcentagem Relatório'
                                                                                ]) \
                                           .set_table_styles([dict(selector="th",props=[('max-width', '90px')
                                                                                      , ('text-align', 'center')])
                                                             ])
df_tot_mes_format = df_tot_mes_format.format(
    {
    "Resultado (R$)": "{:,.2f}".format,
    "Porcentagem": "{:,.2%}".format,
    "Total Lucros": "{:,.2f}".format,
    "Total Perdas": "{:,.2f}".format,
    "Mediana Compras": "{:,.2f}".format,
    "Porcentagem Relatório": "{:,.2%}".format
})

dfFinal['Qtde'] = dfFinal.loc[:, 'Qtde'].astype(int).map('{:,}'.format)
dfFinal["%"] = dfFinal["%"].apply(lambda x: x * 0.01)
dfFinal["%"] = dfFinal["%"].map('{:.2%}'.format)